## The "Generative AI" part of the code

This is where we take the text output of the code and pipe it into GPT

In [1]:
# Install Langchain and openai before running this cell

from langchain.chat_models import AzureChatOpenAI, ChatVertexAI
from langchain.embeddings import OpenAIEmbeddings, VertexAIEmbeddings
from langchain.schema import (
    AIMessage, 
    HumanMessage, 
    SystemMessage
)
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.docstore.document import Document
from langchain.document_loaders import DirectoryLoader
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

import os

import openai

In [2]:
# IF you are running this please just insert the os keys that was given

import sys
sys.path.append('../Gamefication-Retirement/KeysAI')

import OpenAIKeysTIAA

In [3]:
# Opening the file
with open('../Transaction_File_Creator/financial_game_summary.txt', 'r') as f:
    data_to_summarize = f.read()

In [11]:
# Code that was given to us from TIAA to run there model

openai.api_type='azure'
openai.api_base = os.environ["OPENAI_API_HOST"]
openai.api_key = os.environ["OPENAI_API_KEY"]

base_model_name="tiaa-gpt-4-32k"
chosen_llm = AzureChatOpenAI(
        openai_api_base=os.environ["OPENAI_API_HOST"],
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        deployment_name=base_model_name,
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_type="azure",
        temperature=0
    )
embedding_model_name = 'tiaa-text-embedding-ada-002'
chosen_embedding_model = OpenAIEmbeddings(
        deployment=embedding_model_name,
        openai_api_base=os.environ["OPENAI_API_EMBEDDING_HOST"],
        openai_api_version= os.environ["OPENAI_API_VERSION"],
        openai_api_key=os.environ["OPENAI_API_EMBEDDING_KEY"],
        openai_api_type="azure",
        chunk_size=16
    )
finalAnswerTheQuestionModel = AzureChatOpenAI(
        openai_api_base=os.environ["OPENAI_API_HOST"],
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        deployment_name=base_model_name,
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_type="azure",
        max_tokens=4096,
        temperature=0,
        verbose = True
    )
condensingTheQuestionModel = AzureChatOpenAI(
        openai_api_base=os.environ["OPENAI_API_HOST"],
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        deployment_name=base_model_name,
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_type="azure",
        max_tokens=4096,
        temperature=0, 
        verbose = True
    )
memory = ConversationBufferMemory(
    output_key='answer',
    memory_key='chat_history',
    return_messages=True
    )


In [12]:
# we store the data in the vectorstore function for more input 
# also given to us by TIAA

def vectorstore(text):
    new_doc = Document(page_content=text, metadata=[])
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    chunked_docs = text_splitter.split_documents([new_doc])
    vectorstore = FAISS.from_documents(chunked_docs, chosen_embedding_model)
    return(vectorstore)

In [13]:
# storing the output text to vector store so we can input this to gpt 
current_vector = vectorstore(data_to_summarize)

In [14]:
qa = ConversationalRetrievalChain.from_llm(
    llm = finalAnswerTheQuestionModel,
    retriever = current_vector.as_retriever(),
    condense_question_llm = condensingTheQuestionModel,
)
source_with_qa = ConversationalRetrievalChain.from_llm(
    llm = finalAnswerTheQuestionModel,
    retriever = current_vector.as_retriever(
                                         search_kwargs={
                                            "k": 8
                                            }),
    condense_question_llm = condensingTheQuestionModel,
    memory=memory,
    return_source_documents=True,
    verbose=False
)

## Below is the prompting

In [15]:
chat_history = []
query = f"Give A break down for all users and there financial scores, give an in depth overview on why their scores are the way they are."
result = source_with_qa({"question": query, "chat_history": chat_history})
ans = result['answer']

print(ans)

Based on the information provided, there are three users with the following financial scores:

1. Account ID: pznbjQnW6KienZgWVB7EiMQJezZLo1hpqangP
   - Final Score for the month: 68
   - This user lost points for exceeding the essential budget. The exact amount by which the budget was exceeded is not provided. However, the user did score 10 points from savings, indicating some level of financial discipline.

2. Account ID: MAD9LQDKnrC6ybzMWgE9fl6XoRx8eBtL5qyxw
   - Final Score for the month: 68
   - This user also lost points for exceeding the essential budget. The exact amount by which the budget was exceeded is not provided. Like the previous user, this user also scored 10 points from savings.

3. Account ID: 1GErvgE9ANFdK46JejG7iJDVnGjg9qCpaBo7n
   - Final Score for the month: 68
   - This user consistently exceeded their essential budget across multiple months, losing 4 points each time for non-compliance. However, they consistently complied with their non-essential budget, gainin

In [20]:
chat_history = []
query = "Generate a financial review and summary for the account with ID pznbjQnW6KienZgWVB7EiMQJezZLo1hpqangP. Provide the account holder, John, with a detailed breakdown of his score."
result = source_with_qa({"question": query, "chat_history": chat_history})
ans = result['answer']

print(ans)

Sure, here is a financial review and summary for John, who holds the account with ID pznbjQnW6KienZgWVB7EiMQJezZLo1hpqangP:

John started with a base score of 60. He lost 4 points for exceeding the essential budget, but he gained 2 points for complying with the non-essential budget. He also gained an additional 10 points from savings. 

Therefore, his final score for the month is 68. 

In summary, John needs to manage his essential budget better to avoid losing points in the future. However, he is doing well with his non-essential budget and savings.


In [24]:
chat_history = []
query = "respond with a json like response no text response for the user:pznbjQnW6KienZgWVB7EiMQJezZLo1hpqangP with all the elements you find about that user"
result = source_with_qa({"question": query, "chat_history": chat_history})
ansjson = result['answer']

print(ansjson)

{
  "Account ID": "pznbjQnW6KienZgWVB7EiMQJezZLo1hpqangP",
  "Transactions": [
    {
      "Transaction": "United Airlines",
      "Amount": "$500.00",
      "Category": "Non-Essential"
    },
    {
      "Transaction": "Tectra Inc",
      "Amount": "$500.00",
      "Category": "Other"
    },
    {
      "Transaction": "AUTOMATIC PAYMENT - THANK",
      "Amount": "$2078.50",
      "Category": "Essential"
    },
    {
      "Transaction": "KFC",
      "Amount": "$500.00",
      "Category": "Non-Essential"
    },
    {
      "Transaction": "Madison Bicycle Shop",
      "Amount": "$500.00",
      "Category": "Other"
    },
    {
      "Transaction": "Touchstone Climbing",
      "Amount": "$78.50",
      "Category": "Non-Essential"
    }
  ],
  "Month-Year": [
    "2021-11",
    "2022-03",
    "2022-05",
    "2022-02",
    "2022-06",
    "2022-07",
    "2022-01"
  ],
  "Total spending on Essentials": "$2078.50",
  "Total spending on Non-Essentials": "$1078.50",
  "Budget for Essentials": "

## Storing to JSON for the app dev to read

In [26]:
# # dont run if userdata already exists

# json_file_name = 'userdata.json'

# with open(json_file_name, 'a', encoding='utf-8') as file:
#     file.write(ansjson + '\n')